In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from keras.layers.core import Dense, Activation
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


Using TensorFlow backend.


In [3]:
df = pd.read_csv("kddcup.data_10_percent_corrected")

In [6]:
df.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome'
]

#display 5 rows
df[0:5]

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [7]:
#Features
x = df.drop(['outcome'], axis = 1)
y = df[['outcome']]

#Outcomes - Turning into Dummy variables as string cantnot passs through an array into the neural net.
#y = df['outcome']
dummies = pd.get_dummies(y['outcome'])
y = dummies.values

df = pd.DataFrame(x)

df[0:5]


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [8]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [9]:
#Turn categorical data into intergers of 0 & 1 

dummies = pd.get_dummies(df['protocol_type'],['service'],['flag'],['land'], ['logged_in'],['is_host_login'],['is_guest_login'])

#drop as column is now encoded
x = x.drop(['protocol_type','service','flag','land','logged_in','is_host_login','is_guest_login'], axis = 1)

#concat dummy values to the dataframe

x = x.join(dummies)

x[0:5]

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,['service']['flag']tcp,['service']['flag']udp,['service']['flag']nan
0,0,239,486,0,0,0,0,0,0,0,...,0.0,0.05,0.0,0.0,0.0,0.0,0.0,1,0,0
1,0,235,1337,0,0,0,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,1,0,0
2,0,219,1337,0,0,0,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,1,0,0
3,0,217,2032,0,0,0,0,0,0,0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.0,1,0,0
4,0,217,2032,0,0,0,0,0,0,0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.0,1,0,0


In [10]:
# Get column names first
names = x.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit data on the scaler object
scaled_df = scaler.fit_transform(x)
scaled_df = pd.DataFrame(scaled_df, columns=names)

x = pd.DataFrame(scaled_df)

x[0:5]

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,['service']['flag']tcp,['service']['flag']udp,['service']['flag']nan
0,-0.067792,-0.002820,-0.011578,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.146740,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0
1,-0.067792,-0.002824,0.014179,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.188293,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0
2,-0.067792,-0.002840,0.014179,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.188293,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0
3,-0.067792,-0.002842,0.035214,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.209070,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0
4,-0.067792,-0.002842,0.035214,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.209070,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0


In [11]:
#Perform PCA to reduce dimensionality of feature vector for better accuracy
#reduce from 37 to 2 dimensional space

pca=PCA(n_components = 2)
pca.fit(x)
x_scaled = pca.transform(x)
print ("Original Shape:   ", x.shape)
print ("Transformed Shape:",x_scaled.shape)

x = x_scaled

print(x)

Original Shape:    (494020, 37)
Transformed Shape: (494020, 2)
[[0.8696114  0.84916528]
 [0.84788662 0.8285202 ]
 [0.8186531  0.80746397]
 ...
 [0.44228934 0.89046735]
 [0.23680581 1.06179222]
 [0.10792632 1.09605586]]


In [12]:
# Create a test/train split, testing on 25% of data. 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state=42)

In [24]:
from sklearn.neural_network import MLPClassifier

In [14]:
# Measure accuracy
predictors = model.predict(x_test)
predictors = np.argmax(predictors,axis=1)
prediction = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(prediction, predictors)
print("Accuracy:", score * 100,"%")


Accuracy: 99.0939638071333 %


In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(prediction, predictors)

In [20]:
cm

array([[  342,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,   198,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,    11,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     2,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     8,     0,     0,     0,     1,     0,     0,
            0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     3,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    0,     0,     0,     1,     0,   290,     0,     0,     0,
            5,     7,    10,     0

In [22]:
import pickle 
  
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(model) 
  
# Load the pickled model 
#model_kdd = pickle.loads(saved_model) 
  
# Use the loaded pickled model to make predictions 
#model_kdd.predict(X_test) 



In [5]:
df = pd.read_csv("Train_NSL.csv")

In [7]:
d

NameError: name 'd' is not defined